In [1]:
%cd ..

/home/den/dev/git/ozon-e-cup-2025


In [ ]:
from collections import Counter, OrderedDict
from multiprocessing import cpu_count
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm

import src.torch_modules as tm

# Test dataset and dataloader

In [ ]:
class MMDataset(Dataset):
    def __init__(self, data_path: Path, is_test: bool = False):
        self.data_path = data_path
        self.is_test = is_test

    def __len__(self):
        return len(list(self.data_path.iterdir()))

    def __getitem__(self, idx):
        file_path = self.data_path / f"{idx}.npy"
        if self.is_test:
            meta_feats, text_tfidf_feats, img_emb, text_emb = np.load(file_path, allow_pickle=True)
            return meta_feats, text_tfidf_feats, img_emb, text_emb

        meta_feats, text_tfidf_feats, img_emb, text_emb, target = np.load(file_path, allow_pickle=True)
        return meta_feats, text_tfidf_feats, img_emb, text_emb, target

In [ ]:
BATCH_SIZE = 64
N_JOBS = cpu_count()

test_dataset = MMDataset(data_path=Path("data/test"), is_test=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=N_JOBS)

In [ ]:
def to_float(batch):
    return [i.float() for i in batch]

# Load model

In [ ]:
def load_model(model, optimizer=None, scheduler=None, load_path=None):
    checkpoint = torch.load(load_path, weights_only=False)
    model.load_state_dict(checkpoint["model_state_dict"])
    if optimizer is not None and "optimizer_state_dict" in checkpoint:
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    if scheduler is not None and "scheduler_state_dict" in checkpoint:
        scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
    epoch = checkpoint.get("epoch")
    train_losses = checkpoint.get("train_losses")
    val_losses = checkpoint.get("val_losses")
    return epoch, train_losses, val_losses

## MLP

In [ ]:
model = nn.Sequential(
    OrderedDict(
        [
            ("mm_adapter", tm.MultiModalAdapter(input_dims=[102, 100, 512, 384], emb_dim=64, agg="concat")),
            ("mlp", tm.MLPBinaryHead(input_dim=4 * 64)),
        ]
    )
)
load_model(model, None, None, "model_weights/mlp_testing_1_batch.pth")
model.eval()

Sequential(
  (mm_adapter): MultiModalAdapter(
    (proj_layers): ModuleList(
      (0): LinearProjection(
        (linear): Linear(in_features=102, out_features=64, bias=True)
        (norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): LeakyReLU(negative_slope=0.1)
      )
      (1): LinearProjection(
        (linear): Linear(in_features=100, out_features=64, bias=True)
        (norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): LeakyReLU(negative_slope=0.1)
      )
      (2): LinearProjection(
        (linear): Linear(in_features=512, out_features=64, bias=True)
        (norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): LeakyReLU(negative_slope=0.1)
      )
      (3): LinearProjection(
        (linear): Linear(in_features=384, out_features=64, bias=True)
        (norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, a

## Transformer

# Inference

In [ ]:
outputs = []
for batch in tqdm(test_loader):
    batch = to_float(batch)
    with torch.no_grad():
        outputs.append(model(batch))

  0%|          | 0/356 [00:00<?, ?it/s]

# Form submission

In [ ]:
outputs = torch.cat(outputs).squeeze().numpy()

threshold = 0.5
predictions = (outputs > threshold).astype(int)
Counter(predictions)

Counter({np.int64(0): 16274, np.int64(1): 6486})

In [15]:
test_ids = np.load("data/test_ids.npy")
submission = pd.DataFrame({"id": test_ids, "prediction": predictions})
submission.to_csv("submission.csv", index=False)